In [1]:
from ur5 import UR5
ur5 = UR5()

Inicializando a classe UR5...
Inicializando os nós e handles...
Pronto!


In [3]:
from math import pi

In [10]:
ur5.move_to_config([0, 0, 0, 0, 0, 0])

Error: wb_motor_set_velocity() called with negative 'velocity' argument in position control mode (0.0595).
Error: wb_motor_set_velocity() called with negative 'velocity' argument in position control mode (0.0595).
Error: wb_motor_set_velocity() called with negative 'velocity' argument in position control mode (0.01).
Error: wb_motor_set_velocity() called with negative 'velocity' argument in position control mode (-0.8).
Error: wb_motor_set_velocity() called with negative 'velocity' argument in position control mode (0.0595).
Error: wb_motor_set_velocity() called with negative 'velocity' argument in position control mode (-0.8).
Error: wb_motor_set_velocity() called with negative 'velocity' argument in position control mode (0.0595).
Error: wb_motor_set_velocity() called with negative 'velocity' argument in position control mode (0.01).
Error: wb_motor_set_velocity() called with negative 'velocity' argument in position control mode (-0.8).
Error: wb_motor_set_velocity() called with nega

Iterações totais:  157


Error: wb_motor_set_velocity() called with negative 'velocity' argument in position control mode (0.0595).
Error: wb_motor_set_velocity() called with negative 'velocity' argument in position control mode (0.01).
Error: wb_motor_set_velocity() called with negative 'velocity' argument in position control mode (-0.8).
Error: wb_motor_set_velocity() called with negative 'velocity' argument in position control mode (0.0595).
Error: wb_motor_set_velocity() called with negative 'velocity' argument in position control mode (-0.8).
Error: wb_motor_set_velocity() called with negative 'velocity' argument in position control mode (0.0595).
Error: wb_motor_set_velocity() called with negative 'velocity' argument in position control mode (0.01).
Error: wb_motor_set_velocity() called with negative 'velocity' argument in position control mode (-0.8).
Error: wb_motor_set_velocity() called with negative 'velocity' argument in position control mode (0.0595).
Error: wb_motor_set_velocity() called with nega

(5.024000000000001, 89.99426980242826, 45.93849978597246)

In [8]:
ur5.move_to_config([.5*pi, .05*pi, .45*pi, 0, -.5*pi, .5*pi])

Iterações totais:  157


(5.024000000000008, 0.061025542990087285, 0.020190453890403935)

In [9]:
ur5.actuate_gripper(close=0)

Iterações totais:  63
2.015999999999991


In [ ]:
supervisor = Supervisor()

In [ ]:
bottle = supervisor.getFromDef('bottle')

In [ ]:
timestep = int(supervisor.getBasicTimeStep())

In [ ]:
for i in range(supervisor.getNumberOfDevices()):
    print(supervisor.getDeviceByIndex(i).getName())

In [30]:
joint_names = ['shoulder_pan_joint', 'shoulder_lift_joint', 'elbow_joint', 'wrist_1_joint', 'wrist_2_joint', 'wrist_3_joint']
finger_joint_names = ['finger_1_joint_1', 'finger_1_joint_2', 'finger_1_joint_3',
                 'finger_2_joint_1', 'finger_2_joint_2', 'finger_2_joint_3',
                 'finger_middle_joint_1', 'finger_middle_joint_2', 'finger_middle_joint_3']
joints = [supervisor.getDevice(joint) for joint in joint_names]
finger_joints = [supervisor.getDevice(j) for j in finger_joint_names]
finger_joint_limits = [[0.0595, 0.8], [0.01, 1], [-0.8, -0.0623],
                       [0.0595, 0.8], [0.01, 1], [-0.8, -0.0623],
                       [0.0595, 0.8], [0.01, 1], [-0.8,-0.0623]]

for joint in joints:
    print(joint.getName())
    joint.setPosition(float('inf'))
    joint.setVelocity(0)
    joint.setControlPID(1, 0, 0)
    joint.getPositionSensor().enable(timestep)
    print(joint.getPositionSensor().getValue())

for dev in finger_joints:
    dev.getPositionSensor().enable(timestep)
    # print(dev.getPositionSensor().getValue())


shoulder_pan_joint
0.0595000073998063
shoulder_lift_joint
0.019210638020276522
elbow_joint
-0.7996245981777852
wrist_1_joint
0.00026136963855841286
wrist_2_joint
-0.0001407260013864401
wrist_3_joint
-0.7998522373350063


In [31]:
supervisor.step(timestep)

0

In [32]:
def get_joint_angles():
    angles = [joint.getPositionSensor().getValue() for joint in joints]
    angles[0] -= pi
    angles[1] += pi/2
    angles[3] += pi/2
    return angles

In [33]:
def get_finger_angles():
    return [joint.getPositionSensor().getValue() for joint in finger_joints]

In [35]:
get_joint_angles()

[-3.082092645407871,
 1.5900293737067481,
 -0.7996182730605255,
 1.5710585372358825,
 -0.0001416256524865165,
 -0.7998522383408105]

In [36]:
import numpy as np

In [37]:
def move_to_config(target, duration=5):
    t0 = supervisor.getTime()
    v0 = np.zeros(6)
    vf = np.zeros(6)
    q0 = get_joint_angles()
    qf = np.array(target)
    print(q0)
    print(qf)
    a0 = np.zeros(6)
    af = np.zeros(6)
    tf = t0 + duration
    A = np.array([[1, t0, t0**2, t0**3, t0**4, t0**5],
                  [0, 1, 2*t0, 3*t0**2, 4*t0**3, 5*t0**4],
                  [0, 0, 2, 6*t0, 12*t0**2, 20*t0**3],
                  [1, tf, tf**2, tf**3, tf**4, tf**5],
                  [0, 1, 2*tf, 3*tf**2, 4*tf**3, 5*tf**4],
                  [0, 0, 2, 6*tf, 12*tf**2, 20*tf**3]])
    b = np.array([q0, v0, a0, qf, vf, af])
    x = [np.linalg.solve(A, b[:, i]) for i in range(6)]
    time0 = supervisor.getTime()
    iterations = 0
    end_effector_vel = []
    vel_jacob = [[], [], [], [], [], []]
    pos = [[], [], [], [], [], []]
    vel = [[], [], [], [], [], []]
    acc = [[], [], [], [], [], []]
    jerk = [[], [], [], [], [], []]
    time_arr = [[], [], [], [], [], []]
    while supervisor.getTime() <= tf:
        t = supervisor.getTime()
        for joint in joints:
            joint.setVelocity(x[joints.index(joint)][1] +
                              2*x[joints.index(joint)][2]*t +
                              3*x[joints.index(joint)][3]*t**2 +
                              4*x[joints.index(joint)][4]*t**3 +
                              5*x[joints.index(joint)][5]*t**4)


            idx = joints.index(joint)
            p = x[idx][0] + x[idx][1]*t + x[idx][2]*t**2 + x[idx][3]*t**3 + x[idx][4]*t**4 + x[idx][5]*t**5
            v = x[idx][1] + 2*x[idx][2]*t + 3*x[idx][3]*t**2 + 4*x[idx][4]*t**3 + 5*x[idx][5]*t**4
            a = 2*x[idx][2] + 6*x[idx][3]*t + 12*x[idx][4]*t**2 + 20*x[idx][5]*t**3
            j = 6*x[idx][3] + 24*x[idx][4]*t + 60*x[idx][5]*t**2
            time_arr[idx].append(t-time0)
            pos[idx].append(p)
            vel[idx].append(v)
            acc[idx].append(a)
            jerk[idx].append(j)
            v = x[idx][1] + 2*x[idx][2]*t + 3*x[idx][3]*t**2 + 4*x[idx][4]*t**3 + 5*x[idx][5]*t**4
            vel_jacob[idx].append(v)
        supervisor.step(timestep)
        iterations += 1
    for joint in joints:
        joint.setVelocity(0)
    timef = supervisor.getTime()
    error = np.abs(np.array(target) - get_joint_angles())*180/np.pi
    print('Iterações totais: ', iterations)
    print(timef-time0)
    print(np.max(error))
    print(np.mean(error))

In [39]:
def actuate_gripper(close=0):
    t0 = supervisor.getTime()
    v0 = np.zeros(9)
    vf = np.zeros(9)
    q0 = get_finger_angles()
    qf = np.hstack((np.array([lim[close] for lim in finger_joint_limits])))
    a0 = np.zeros(9)
    af = np.zeros(9)
    tf = t0 + 2
    A = np.array([[1, t0, t0**2, t0**3, t0**4, t0**5],
                  [0, 1, 2*t0, 3*t0**2, 4*t0**3, 5*t0**4],
                  [0, 0, 2, 6*t0, 12*t0**2, 20*t0**3],
                  [1, tf, tf**2, tf**3, tf**4, tf**5],
                  [0, 1, 2*tf, 3*tf**2, 4*tf**3, 5*tf**4],
                  [0, 0, 2, 6*tf, 12*tf**2, 20*tf**3]])
    b = np.array([q0, v0, a0, qf, vf, af])
    x = [np.linalg.solve(A, b[:, i]) for i in range(9)]
    time0 = supervisor.getTime()
    iterations = 0
    while supervisor.getTime() <= tf:
        t = supervisor.getTime()
        for idx, joint in enumerate(finger_joints):
            joint.setPosition(x[idx][0] + x[idx][1]*t + x[idx][2]*t**2 + x[idx][3]*t**3 + x[idx][4]*t**4 + x[idx][5]*t**5)
        supervisor.step(timestep)
        iterations += 1
    for i, joint in enumerate(joints):
        joint.setPosition(finger_joint_limits[i][close])
    timef = supervisor.getTime()
    print('Iterações totais: ', iterations)
    print(timef-time0)

In [41]:
actuate_gripper(1)

Iterações totais:  63
2.016000000000002


In [ ]:
# bottle.getField('translation').setSFVec3f([-1.467559345817077, 1.7836801495521285e-06, 0.9004484925267724])

In [ ]:
bottle.getField('translation').getSFVec3f()

In [ ]:
supervisor.step(timestep)

In [42]:
move_to_config([0, 0, 0, 0, 0, 0])

[-3.0820927252875077, 1.5928526768789613, -0.7988206061594614, 1.5711641710936382, -0.0002556565399703571, -0.7998522955284084]
[0 0 0 0 0 0]


Error: wb_motor_set_velocity() called with negative 'velocity' argument in position control mode (0.8).
Error: wb_motor_set_velocity() called with negative 'velocity' argument in position control mode (-0.0623).
Error: wb_motor_set_velocity() called with negative 'velocity' argument in position control mode (0.8).
Error: wb_motor_set_velocity() called with negative 'velocity' argument in position control mode (1).
Error: wb_motor_set_velocity() called with negative 'velocity' argument in position control mode (1).
Error: wb_motor_set_velocity() called with negative 'velocity' argument in position control mode (0.8).
Error: wb_motor_set_velocity() called with negative 'velocity' argument in position control mode (1).
Error: wb_motor_set_velocity() called with negative 'velocity' argument in position control mode (0.8).
Error: wb_motor_set_velocity() called with negative 'velocity' argument in position control mode (1).
Error: wb_motor_set_velocity() called with negative 'velocity' argum

Iterações totais:  157
5.024000000000001
134.7687050504913
54.62366217960001


Error: wb_motor_set_velocity() called with negative 'velocity' argument in position control mode (1).
Error: wb_motor_set_velocity() called with negative 'velocity' argument in position control mode (0.8).
Error: wb_motor_set_velocity() called with negative 'velocity' argument in position control mode (1).
Error: wb_motor_set_velocity() called with negative 'velocity' argument in position control mode (0.8).


In [21]:
move_to_config([.5*pi, .05*pi, .45*pi, 0, -.5*pi, .5*pi])

Error: wb_motor_set_velocity() called with negative 'velocity' argument in position control mode (0.0595).
Error: wb_motor_set_velocity() called with negative 'velocity' argument in position control mode (0.01).
Error: wb_motor_set_velocity() called with negative 'velocity' argument in position control mode (0.0595).
Error: wb_motor_set_velocity() called with negative 'velocity' argument in position control mode (0.01).
Error: wb_motor_set_velocity() called with negative 'velocity' argument in position control mode (0.01).
Error: wb_motor_set_velocity() called with negative 'velocity' argument in position control mode (0.0595).
Error: wb_motor_set_velocity() called with negative 'velocity' argument in position control mode (0.01).
Error: wb_motor_set_velocity() called with negative 'velocity' argument in position control mode (0.01).
Error: wb_motor_set_velocity() called with negative 'velocity' argument in position control mode (0.0595).
Error: wb_motor_set_velocity() called with nega

Iterações totais:  157
5.024000000000001
266.59089590593646
131.48055111518536


Error: wb_motor_set_velocity() called with negative 'velocity' argument in position control mode (0.01).
Error: wb_motor_set_velocity() called with negative 'velocity' argument in position control mode (0.0595).
Error: wb_motor_set_velocity() called with negative 'velocity' argument in position control mode (0.01).
Error: wb_motor_set_velocity() called with negative 'velocity' argument in position control mode (0.01).
Error: wb_motor_set_velocity() called with negative 'velocity' argument in position control mode (0.0595).
Error: wb_motor_set_velocity() called with negative 'velocity' argument in position control mode (0.01).
Error: wb_motor_set_velocity() called with negative 'velocity' argument in position control mode (0.01).
Error: wb_motor_set_velocity() called with negative 'velocity' argument in position control mode (0.0595).
Error: wb_motor_set_velocity() called with negative 'velocity' argument in position control mode (0.01).


In [ ]:
move_to_config([-.5*pi, -.05*pi, -.45*pi, 0, .5*pi, .5*pi])

In [ ]:
move_to_config([pi/2, 0, 0, 0, 0, 0])

In [ ]:
move_to_config([.1*pi, .18*pi, .57*pi, 0.25*pi, 0.2, pi/2])

In [ ]:
move_to_config([-.3*pi, -.04*pi, -.8*pi, -0.17*pi, -0.5, 0])

In [ ]:
move_to_config([0.2, .04*pi, .8*pi, 0.17*pi, 0.3, pi/2])